# Try the RM demo with some modifications:
1. pyvista for plotting
2. parallel with ipyparallel

In [2]:
import ipyparallel as ipp

# create a cluster
cluster = ipp.Cluster(engines="mpi", n=2)
# start that cluster and connect to it
rc = cluster.start_and_connect_sync(activate=True)

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

In [3]:
%%px

import numpy as np

import dolfinx
import ufl
from dolfinx.fem import Function, FunctionSpace, dirichletbc, Constant
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import CellType, create_unit_square
from ufl import (FiniteElement, MixedElement, VectorElement, dx, grad, inner,
                 split, sym, tr)

from mpi4py import MPI

## Create mesh

In [4]:
%%px

mesh = create_unit_square(MPI.COMM_WORLD, 32, 32, CellType.quadrilateral)

## Define Elements, FunctionSpace, Functions 

In [5]:
%%px

U_el = MixedElement([VectorElement("Lagrange", ufl.quadrilateral, 1), FiniteElement("Lagrange", ufl.quadrilateral, 1),
                     FiniteElement("RTCE", ufl.quadrilateral, 1), FiniteElement("RTCE", ufl.quadrilateral, 1)])
U = FunctionSpace(mesh, U_el)

u_ = Function(U)
u = ufl.TrialFunction(U)
u_t = ufl.TestFunction(U)

theta_, w_, R_gamma_, p_ = split(u_)

## Material parameters

In [6]:
%%px

from dolfinx import default_scalar_type

E = Constant(mesh, default_scalar_type(10920.0))
nu = Constant(mesh, default_scalar_type(0.3))
kappa = Constant(mesh, default_scalar_type(5.0 / 6.0))
t = Constant(mesh, default_scalar_type(0.001))

## Bending strain and bending energy

Bending strain tensor k:
$$
 k(\theta) = \dfrac{1}{2}(\nabla \theta + (\nabla \theta)^T)
$$

Bending energy $\psi_b$ and bending modulus 
$$
 \psi_b(k) = \frac{1}{2} D \left( (1 - \nu) \, \mathrm{tr}\,(k^2) + \nu \, (\mathrm{tr}    \,k)^2 \right) \qquad
 D = \frac{Et^3}{12(1 - \nu^2)}
$$

In [7]:
%%px

k = sym(grad(theta_)) 

D = (E*t**3)/(12.0*(1.0 - nu**2))

psi_b = 0.5 * D*((1.0 - nu)*tr(k*k) + nu*(tr(k))**2)

## Shear energy

In [8]:
%%px

psi_s = ((E*kappa*t)/(4.0*(1.0 + nu)))*inner(R_gamma_, R_gamma_)

## External energy

In [9]:
%%px

W_ext = inner(1.0*t**3, w_)*dx

## Imposing MITC method weekly

In [10]:
%%px

gamma = grad(w_) - theta_

# internal Facets
dSp = ufl.Measure('dS', metadata={'quadrature_degree': 1}) 

# boundary Facets
dsp = ufl.Measure('ds', metadata={'quadrature_degree': 1})

# normal of the facet
n = ufl.FacetNormal(mesh)

# tangent of the facet
t = ufl.as_vector((-n[1], n[0]))

# imposing MITC conditions weekly with lagrange multipliers
# (+) is the DG operator
def inner_e(x, y):
    return (inner(x, t)*inner(y, t))('+') * \
        dSp + (inner(x, t)*inner(y, t))*dsp


Pi_R = inner_e(gamma - R_gamma_, p_)

## Jacobian and residual

In [11]:
%%px

Pi = psi_b*dx + psi_s*dx + Pi_R - W_ext
F = ufl.derivative(Pi, u_, u_t)
J = ufl.derivative(F, u_, u)

## Boundary conditions : all clamped

In [12]:
%%px

# x with shape (gdim, num_points)
def all_boundary(x):
    return np.full(x.shape[1], True, dtype=bool)

def left_or_right(x):
    return np.logical_or(np.isclose(x[0], 0.0), np.isclose(x[0], 1.0))


def top_or_bottom(x):
    return np.logical_or(np.isclose(x[1], 0.0), np.isclose(x[1], 1.0))

def make_bc(value, V, on_boundary):
    boundary_entities = dolfinx.mesh.locate_entities_boundary(
        mesh, mesh.topology.dim - 1, on_boundary)
    boundary_dofs = dolfinx.fem.locate_dofs_topological(
        V, mesh.topology.dim - 1, boundary_entities)
    bc = dirichletbc(value, boundary_dofs, V)
    return bc


bcs = []
# Transverse displacements fixed everywhere
bcs.append(make_bc(np.array(0.0, dtype=np.float64), U.sub(1), all_boundary))

# First component of rotation (the rotation about y axis) fixed on top and bottom
bcs.append(make_bc(np.array(0.0, dtype=np.float64), U.sub(0).sub(0), top_or_bottom)) 

# Second component of rotation (the rotation about x axis) fixed on left and right
bcs.append(make_bc(np.array(0.0, dtype=np.float64), U.sub(0).sub(1), left_or_right))


## Solving with Petsc 

In [13]:
%%px

problem = LinearProblem(J, -F, bcs=bcs, petsc_options={
                        "ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
u_ = problem.solve()

theta, w, R_gamma, p = u_.split()

## Testing the results

In [14]:
%%px

# bb_tree is a kind of data structure to quickly find the cells 
# that are close to a given point
bb_tree = dolfinx.geometry.bb_tree(mesh, 2)

# target point
point = np.array([[0.5, 0.5, 0.0]], dtype=np.float64)

# Find the leaf that the target point is in
cell_candidates = dolfinx.geometry.compute_collisions_points(bb_tree, point)

# Find the cell from the leaf that contains the target point
cells = dolfinx.geometry.compute_colliding_cells(
    mesh, cell_candidates, point)

if len(cells) > 0:
    value = w.eval(point, cells.array[0])
    print(value[0])

[stdout:0] 4.061094603185817e-06


[stdout:1] 4.061094603185817e-06


## Write results to files

In [15]:
%%px

from dolfinx import io
from pathlib import Path

results_folder = Path("results/RM_simply_supported")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "RM_simply_supported"

with io.VTXWriter(mesh.comm, filename.with_suffix(".bp"), [w]) as vtx:
    vtx.write(0.0)

with io.XDMFFile(mesh.comm, filename.with_suffix(".xdmf"), "w") as f:
    f.write_mesh(mesh)
    f.write_function(w)

## Plotting with pyvista

In [16]:
%%px

import pyvista
from dolfinx import plot

#pyvista.start_xvfb()
W,_ = U.sub(1).collapse()

topology, cell_types, geometry = plot.vtk_mesh(W)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
w_collapsed = w.collapse()
grid.point_data["w"] = w_collapsed.x.array
grid.set_active_scalars("w")

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()


%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

[output:0]

Widget(value='<iframe src="http://localhost:45829/index.html?ui=P_0x7f71b981de50_0&reconnect=auto" class="pyvi…

[output:1]

Widget(value='<iframe src="http://localhost:40511/index.html?ui=P_0x7f841d8f9970_0&reconnect=auto" class="pyvi…

In [17]:
%%px

warped = grid.warp_by_scalar("w", factor=100000)
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)

plotter2.show()


[output:1]

Widget(value='<iframe src="http://localhost:40625/index.html?ui=P_0x7fced72e6cc0_1&reconnect=auto" class="pyvi…

[output:0]

Widget(value='<iframe src="http://localhost:35289/index.html?ui=P_0x7f7cf5e05d60_1&reconnect=auto" class="pyvi…

In [17]:
cluster.stop_cluster_sync()

Stopping controller
Stopping engine(s): 1714137118


Controller stopped: {'exit_code': 0, 'pid': 1890882, 'identifier': 'ipcontroller-1714137117-00j5-1890704'}
engine set stopped 1714137118: {'exit_code': 1, 'pid': 1890938, 'identifier': 'ipengine-1714137117-00j5-1714137118-1890704'}
